In [18]:
from datasets import load_dataset
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

In [19]:
dataset = load_dataset("cats_vs_dogs")
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 23410
    })
})

In [20]:
import os

exp_config = dict()
main_dir = './dataset'
os.makedirs(main_dir, exist_ok=True)

mydataset = pd.DataFrame(columns=['image_path', 'label'])

for i in range(len(dataset['train'])):
    img_path = f"{main_dir}/img_{i}.jpeg"
    
    if not os.path.exists(img_path):
        dataset['train'][i]['image'].save(img_path)
    
    mydataset.at[i, 'image_path'] = img_path
    mydataset.at[i, 'label'] = dataset['train'][i]['labels']

mydataset.head()

,image_path,label
0,./dataset/img_0.jpeg,0
1,./dataset/img_1.jpeg,0
2,./dataset/img_2.jpeg,0
3,./dataset/img_3.jpeg,0
4,./dataset/img_4.jpeg,0


In [21]:
seed = 24
test_size = 0.20
val_size = 0.20

exp_config['seed'] = seed
exp_config['test_size'] = test_size
exp_config['val_size'] = val_size

In [22]:
train_val_df, test_df = train_test_split(mydataset, test_size=test_size, stratify=mydataset['label'], random_state=seed)

train_df, val_df = train_test_split(train_val_df, test_size=val_size, stratify=train_val_df['label'], random_state=seed)

In [23]:
exp_config['train_n_cats'] = train_df['label'].value_counts()[0]
exp_config['train_n_dogs'] = train_df['label'].value_counts()[1]
exp_config['val_n_cats'] = val_df['label'].value_counts()[0]
exp_config['val_n_dogs'] = val_df['label'].value_counts()[1]
exp_config['test_n_cats'] = test_df['label'].value_counts()[0]
exp_config['test_n_dogs'] = test_df['label'].value_counts()[1]

In [24]:
class CatsDogsDataset(Dataset):
    def __init__(self, img_path_list, lab_list, transform=None):
        self.transform = transform
        self.images = img_path_list
        self.labels = lab_list

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")

        label = self.labels[idx]
        label = torch.Tensor([label])

        if self.transform:
            image = self.transform(image)

        return image, label

In [25]:
input_size = (224,224)
exp_config['input_size'] = input_size

In [26]:
n_channels = 3
exp_config['n_channels'] = n_channels

In [27]:
transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.ToTensor(),
])

In [28]:
train_dataset = CatsDogsDataset(train_df['image_path'].tolist(), train_df['label'].tolist(), transform)
val_dataset = CatsDogsDataset(val_df['image_path'].tolist(), val_df['label'].tolist(), transform)
test_dataset = CatsDogsDataset(test_df['image_path'].tolist(), test_df['label'].tolist(), transform)

In [29]:
from torch.utils.data import DataLoader

batch_size = 64
exp_config['batch_size'] = batch_size

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=False)

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F

class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 1)

    def forward(self, x):
        x = self.base_model(x)
        x = torch.sigmoid (x)
        return x

class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34, self).__init__()
        self.base_model = models.resnet34(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 1)

    def forward(self, x):
        x = self.base_model(x)
        x = torch.sigmoid(x)
        return x

class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 1)

    def forward(self, x):
        x = self.base_model(x)
        x = torch.sigmoid(x)
        return x

In [31]:
if os.name == 'posix':
    device = torch.device("mps" if torch.mps.is_available() else "cpu")
elif os.name == 'nt':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
    
device

device(type='mps')

In [32]:
# model18 = ResNet18.to(device)
# model34 = ResNet34.to(device)
# model50 = ResNet50.to(device)